## Python Exercise 4 - 6

**Ingrid Gustavsen**

Deadline: **30/03**

### Exercise 4: The Bragg_Williams approximation

**TASK 1 - Finding the exact density of states**

- density_of_states.py and the density of states pngs
- The variance decreases as the system size increases. This is because the number of microstates increases as the system size increases.
- The program uses .. seconds to find all possible configurations for a system with 24 particles. 
- The multiplication of the number of microstates increases exponentially with the system size.
- This is why it is not possible to use this method on larger systems.

In [1]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import time

In [2]:
def get_lattice_shape(n): 
    """
    Determines the lattice shapes that will be used in this exercise, given n particles.
    """
    shapes = {4:(2,2), 6:(2,3), 8:(2,4), 12:(3,4), 16:(4,4), 20:(5,4), 24:(6,4)}
    return shapes[n]

In [3]:
def count_AB(lattice): 
    """
    Returns the number of AB-interactions, n_AB, for a provided lattice.
    Uses periodic boundary conditions.
    """
    n_AB = 0

    for coord, particle in np.ndenumerate(lattice): # coord is the coordinate, particle is the value at that coordinate, ndenumerate gives both
    
        if particle == 'A': #Prevents double counting
            # loop over the number of possible neighbours
            for incr in [(1,0),(0,-1),(-1,0),(0,1)]: # neighbour increments
                # The modulo operation ensures that periodic boundary conditions are employed
                # neighbour needs to be a tuple
                neighbour = np.array(coord) + np.array(incr) #add the coordinate to the increment
                neighbour = tuple(np.mod(neighbour, get_lattice_shape(lattice.size))) # ADD your code
                if lattice[neighbour] == 'B': #Checks if AB-interaction
                    n_AB += 1 
    return n_AB

In [4]:
def create_arrays_and_count(n):  #takes the number of particles as input
    """
    Given the number of molecules/lattice points, n, and returns a list with the number of AB interactions for all microstates, m_AB.
    A condition is that the number of A and B molecules is equal.
    """
    m_AB = [] # List to store the number of AB interactions for each microstate
    for config in multiset_permutations(int(n/2)*'AB'): 
        lattice = np.reshape(np.array(config), get_lattice_shape(n)) # Here, convert config to a numpy array and convert to a lattice (using np.reshape and lattice_shape above) 
        m_AB.append(count_AB(lattice)) # Then count the number of AB interactions using the function count_A
    return m_AB

In [5]:
for i in [4, 6, 8, 12, 16, 20, 24]: # The system sizes, shorten the list while testing 
    
    start = time.time() #Starts timer
    m_AB = create_arrays_and_count(i)
    end = time.time()
    print('Time elapsed for %i particles: %6.4f seconds' % (i, end-start))

    macrostates = sorted(list(set(m_AB))) # Sorts the macrostates list in ascending order 
    degeneracies = [m_AB.count(macrostate) for macrostate in macrostates] # Counts the number of times a certain number of bonds appear in m_AB 
    #Plots bar chart
    y_pos = np.arange(len(macrostates))
    print(y_pos)

    plt.figure(figsize=(10, 6))
    sns.barplot(x=y_pos, y=degeneracies) #plt.bar(y_pos, degeneracies)
    plt.title('Density of states for %i particles' % i)
    plt.xlabel("number of interactions AB", rotation=0)
    plt.ylabel("number of microstates")
    plt.xticks(y_pos, macrostates, fontsize=7, rotation=30)
    plt.savefig('density_of_states' + str(i))
    plt.clf()
    #Calculates the variance of the microstate list. The variance decreases as system size increases because the system becomes more ordered.
    normalization_factor = max(m_AB) # The interaction energies are normalized
    normalized_mAB = [round(norm/normalization_factor, 2) for norm in m_AB] #Performs normalization
    print('%i particles:\nVariance: %6.4f\n' % (i, np.var(normalized_mAB))) #Prints variance

with open('m_AB.pkl', 'wb') as f: #Save microstate list along with system size for i = i_max (24 in this case)
    pickle.dump([m_AB,i], f)

Time elapsed for 4 particles: 0.0011 seconds
[0 1]
4 particles:
Variance: 0.0556

Time elapsed for 6 particles: 0.0030 seconds
[0 1]
6 particles:
Variance: 0.0336

Time elapsed for 8 particles: 0.0072 seconds
[0 1 2 3 4]
8 particles:
Variance: 0.0198

Time elapsed for 12 particles: 0.1868 seconds
[0 1 2 3 4 5 6]
12 particles:
Variance: 0.0116

Time elapsed for 16 particles: 3.2157 seconds
[0 1 2 3 4 5 6 7 8 9]
16 particles:
Variance: 0.0063

Time elapsed for 20 particles: 48.4887 seconds
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
20 particles:
Variance: 0.0068

Time elapsed for 24 particles: 859.1947 seconds
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
24 particles:
Variance: 0.0045



<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

**TASK 2 - Determine the equilibrium degree of mixing**

We define the degree of mixing as the number of AB-interactions. To find the equilibrium degree, plot the
Helmholtz free energy in eq. 2 as a function of Ui for varying values of T ranging from 1-10. Consider only
systems with 24 particles. 

- show helmholtz_free_energy.png
- As the temperature increases, the equilibrium degree of mixing decreases.
- What is the physical explanation?


In [8]:
import numpy as np
from math import factorial, sqrt, exp, log
import matplotlib.pyplot as plt
import pickle

def calculate_F(m_AB):
    """
    Plots the Helmholtz free energy as a function of the energy for different temperatures.
    Given is a list of microstates, m_AB
    """
    macrostates = sorted(list(set(m_AB))) # Get macrosteps. Sorts list in ascending order
    degeneracies = [m_AB.count(macrostate) for macrostate in macrostates]
    temperatures = np.linspace(0,4,10) # the temperature range 
    for temp in temperatures:
        # Calculate a list with free energies. Use list comprehension. Hint: use zip(macrostates, degeneracies) as iterator
        free_energies =  []

        for m, w in zip(macrostates, degeneracies):
            free_energies.append(-m - temp*log(w)) # W = microstates * degeneracies. U = -m, m = 24. T = temp. F = U - T*ln(W)
        
        plt.plot(macrostates, free_energies)
        plt.xlabel("number of macrostates", rotation=0)
        plt.ylabel("free energy")
    plt.savefig('helmholtz_free_energy') #Saves plot as Helmholtz_free_energy.png
    plt.close()

with open('m_AB.pkl', 'rb') as f: # Loads microstate list
    m_AB = pickle.load(f)[0]
calculate_F(m_AB)


**TASK 3 - Comparing the exact and approximate partition functions**

- Q: Partition function of temperature
- The approximation is not very good at low temperatures, because the multilplicity is very small
- As the temperature increases, the partition function increases. The exact partition function increases faster than the approximate partition function. The ratio of the exact and approximate partition function approaches 1 as the temperature increases.

In [ ]:
import numpy as np
from math import factorial, sqrt, exp, log
import sympy as sym
import matplotlib.pyplot as plt
import pickle

def calculate_Q_exact(m_AB, temp_inv) -> float: #returns float
    """
    Calculates the partition function as a sum of a list with Boltzmann factors.
    Given is a list with AB-bonds, m_AB, providing the energy (a minus sign difference)
    and the inverse of the temperature, temp_inv. The Boltzmann constant is here set to 1.
    """

    macrostates = sorted(list(set(m_AB))) # Get macrosteps. Sorts list in ascending order

    v: list = [exp(N*temp_inv) for N in macrostates]
    return sum(v)

def calculate_Q_approx(n, temp):
    """
    Calculates the partition function using the Bragg-Williams approximation
    Gives the number of particles, n, and the temperature, temp.
    Get the multiplicity (not from Stirling's approximation!)
    Returns ln Q
    """
    multiplicity = factorial(n)/(factorial(n//2))**2
    approx = n/temp + log(multiplicity)
    return approx


def partition_function_plot(m_AB, n) -> float:
    """
    Generates plots with the partition function.
    Given is a list with AB-bonds, m_AB, providing the energy (a minus sign difference)
    and the number of particles, n.
    """
    temps = np.linspace(1,10,100) #Temperature range to be plotted. High resolution
    Q_exact_list = []
    Q_approx_list = []
    Q_ratio = []
    #The for loop calculates and appends Q(T)
    for temp in temps:
        Q_exact = calculate_Q_exact(m_AB, 1.0/temp)
        Q_approx = calculate_Q_approx(n, temp)
        Q_exact_list.append(log(Q_exact))
        Q_approx_list.append(Q_approx)
        Q_ratio.append(log(Q_exact)/Q_approx)
    #Makes and saves plots
    plt.plot(temps, Q_exact_list)
    plt.plot(temps, Q_approx_list)
    plt.legend(['ln Q_exact', 'ln Q_approx'], loc='upper left')
    plt.savefig('partition_function')
    plt.close()
    plt.plot(temps, Q_ratio)
    plt.xlabel("Temperature", rotation=0)
    plt.ylabel("ln Q_exact/Q_approx")
    plt.legend(['Q_ratio'], loc='upper left')
    plt.savefig('partition_function_ratio')
    plt.close()

with open('m_AB.pkl', 'rb') as f: #Opens microstate list
    m_AB, n = pickle.load(f) #and creates variables m_AB (list) and N (system size)

partition_function_plot(m_AB, n)


### Exercise 5: Kinetic Monte Carlo simulations

- Start with NA = NB = 1000 and NC = 0. Let this simulation run for 10000 steps. Can you identify, just by looking at the plot output, at what time the system reached equilibrium?
- Run the program again, but with different values for NA, NB, and NC . Let NA ̸= NB. Run these simulations until equilibrium is reached, instead of using a given number of steps. What did you use as an equilibrium condition? Why is this valid?
- Lastly, change the parameter kf and kr relative to each other. How does this effect the result (refer to the relation between equilibrium constant and rate constants in eq. 19.5 in Dill)?